In [1]:
#импорт библиотек
import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from sklearn.metrics import f1_score

import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import optuna

%matplotlib inline
# plt.style.use('seaborn')

data = pd.read_csv('data/biologic.csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [2]:
X = data.drop(columns='Activity')
y = data['Activity']

random_state = 42

X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, random_state=random_state)

# Логистическая регрессия

## Baseline

In [3]:
log_reg = linear_model.LogisticRegression(random_state=random_state, max_iter=1000)

kf = model_selection.KFold(n_splits=5)

cv_metrics = model_selection.cross_validate(
    estimator=log_reg,
    X=X_train,
    y=y_train,
    cv=kf,
    scoring='f1',
    return_train_score=True
)

print('Train k-fold mean f1 score: {:.2f}'.format(np.mean(cv_metrics['train_score'])))
print('Valid k-fold mean f1 score: {:.2f}'.format(np.mean(cv_metrics['test_score'])))

Train k-fold mean f1 score: 0.90
Valid k-fold mean f1 score: 0.77


Train k-fold mean f1 score: 0.89

Valid k-fold mean f1 score: 0.78

## GRIDSEARCHCV

In [4]:
param_grid = [
              { 
               'C': np.logspace(-3, 3, 7),
                'penalty': ['l2'],
                'solver': ['liblinear', 'saga', 'lbfgs', 'newton-cg']
              },
              
              {
                'penalty': [None],
                'solver': ['saga', 'lbfgs', 'newton-cg']
              },
              
              { 
               'C': np.logspace(-3, 3, 7),
                'penalty': ['l1'],
                'solver': ['liblinear', 'saga']
              },
             ]

In [5]:
grid_search = model_selection.GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=random_state, max_iter=1000),
    param_grid=param_grid,
    cv=5,
    n_jobs=-1
)

%time grid_search.fit(X_train, y_train) 
y_test_pred = grid_search.predict(X_valid)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 4.02 s
Wall time: 5min 28s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}


CPU times: total: 4.02 s

Wall time: 5min 28s

f1_score на тестовом наборе: 0.79

Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}

Wall time: 5min 8s

f1_score на тестовом наборе: 0.79

Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}

## RANDOMIZEDSEARCHCV

In [6]:
random_search = model_selection.RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=random_state, max_iter=1000),
    param_distributions=param_grid,
    cv=5,
    n_iter=20,
    n_jobs=-1
)

%time random_search.fit(X_train, y_train) 
y_test_pred = random_search.predict(X_valid)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: total: 3.56 s
Wall time: 2min 38s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.1}


CPU times: total: 3.56 s

Wall time: 2min 38s

f1_score на тестовом наборе: 0.79

Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.1}

## HYPEROPT

In [19]:
space = {
    'C': hp.loguniform('C', -3, 3),
    'penalty': hp.choice('penalty', [None, 'l2']),
    'solver': hp.choice('solver', ['saga', 'lbfgs', 'newton-cg'])
}

In [20]:
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    if params['penalty'] is None:
        penalty = None
    else:
        penalty = str(params['penalty'])

    params = {
              'C': float(params['C']),
              'penalty': penalty, 
              'solver': str(params['solver'])
              }
  
    model = linear_model.LogisticRegression(**params, random_state=random_state, max_iter=1000)
    
    # применим  cross validation с тем же количеством фолдов
    score = model_selection.cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return -score

In [21]:
%%time

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 20/20 [05:46<00:00, 17.31s/trial, best loss: -0.7865463231251131]
Наилучшие значения гиперпараметров {'C': 0.08890986938340578, 'penalty': 1, 'solver': 0}
CPU times: total: 797 ms
Wall time: 5min 46s


Наилучшие значения гиперпараметров {'C': 0.08890986938340578, 'penalty': 1, 'solver': 0}

CPU times: total: 797 ms

Wall time: 5min 46s

Параметры сохранились некорректно, попробуем подобрать параметры для модели, взяв их по индексам из списков из пространства параметров

In [22]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(
    random_state=random_state, 
    C=float(best['C']),
    penalty='l2',
    # penalty=str(best['penalty']),
    # solver=str(best['solver'])
    solver='saga'
)
model.fit(X_train, y_train)
y_valid_pred = model.predict(X_valid)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))

f1_score на тестовом наборе: 0.79


c:\Users\oxi-a\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## OPTUNA

In [11]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  solver = trial.suggest_categorical('solver', ['saga', 'lbfgs', 'newton-cg'])
  penalty =  trial.suggest_categorical('penalty', [None, 'l2'])
  C = trial.suggest_float('C', 0.001, 1000)

  # создаем модель
  model = linear_model.LogisticRegression(max_iter=1000,
                                          penalty=penalty,
                                          solver=solver,
                                          C=C)
  
  kf = model_selection.KFold(n_splits=5)

  cv_metrics = model_selection.cross_validate(
    estimator=model,
    X=X_train,
    y=y_train,
    cv=kf,
    scoring='f1',
    return_train_score=True
  )

  return np.mean(cv_metrics['test_score'])

In [12]:
%%time

study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
study.optimize(optuna_rf, n_trials=10)

[I 2024-02-28 16:55:27,279] A new study created in memory with name: LogisticRegression
c:\Users\oxi-a\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\oxi-a\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\oxi-a\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\oxi-a\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\oxi-a\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_

CPU times: total: 21min 20s
Wall time: 12min 48s


CPU times: total: 21min 20s

Wall time: 12min 48s

In [13]:
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

model = linear_model.LogisticRegression(**study.best_params, random_state=random_state)
model.fit(X_train, y_train)
y_valid_pred = model.predict(X_valid)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))

c:\Users\oxi-a\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1186: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Наилучшие значения гиперпараметров {'solver': 'saga', 'penalty': None, 'C': 691.1733513735513}
f1_score на обучающем наборе: 0.77
f1_score на тестовом наборе: 0.78


c:\Users\oxi-a\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Наилучшие значения гиперпараметров {'solver': 'saga', 'penalty': None, 'C': 691.1733513735513}

f1_score на обучающем наборе: 0.77

f1_score на тестовом наборе: 0.78

## Логистическая регрессия - Итог

Методы от самого быстрого до самого долгого: RandomizedSearchCV, GridSearchCV, hyperopt, optuna.

Метрика для базовой модели была 0.77, метрики при подборе гиперпараметров повысились до 0.79. При max_evals=30 у "hyperopt" метрика повышалась до 0.8, но так надо дольше ждать, пока подберёт параметры. Метрика от "optuna" могла бы быть выше, т.к. значение n_trials мало, но даже с таким значением подбор занимает много времени.

Стоит отметить, что проще всего реализовать было первые два метода т.к. у логистической регрессии есть ряд параметров, которые конфликтуют между собой, такие как penalty и solver. Для "поиска по сетке" и "случайного поиска" можно легко задать зваимоисключающие параметры разными наборами параметров. У "hyperopt" и "optuna" с этим сложнее.

# Случайный лес

## baseline

In [6]:
ran_for = ensemble.RandomForestClassifier(
    n_estimators=500,
    criterion='entropy',
    max_depth=8,
    max_features='sqrt',
    random_state=random_state
)

kf = model_selection.KFold(n_splits=5)

cv_metrics = model_selection.cross_validate(
    estimator=ran_for,
    X=X_train,
    y=y_train,
    cv=kf,
    scoring='f1',
    return_train_score=True
)

print('Train k-fold mean f1 score: {:.2f}'.format(np.mean(cv_metrics['train_score'])))
print('Valid k-fold mean f1 score: {:.2f}'.format(np.mean(cv_metrics['test_score'])))

Train k-fold mean f1 score: 0.90
Valid k-fold mean f1 score: 0.79


Train k-fold mean f1 score: 0.90

Valid k-fold mean f1 score: 0.79

## GRIDSEARCHCV

In [15]:
param_grid = [
              { 
               'n_estimators': [50, 100, 250, 500],
                'criterion': ['gini', 'entropy'],
                'max_depth': [3, 5, 8],
                'min_samples_leaf': [1, 3, 5],
                'max_features': ['sqrt', 'log2', None]
              }
             ]

In [16]:
grid_search = model_selection.GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=random_state),
    param_grid=param_grid,
    cv=5,
    n_jobs=-1
)

%time grid_search.fit(X_train, y_train) 
y_test_pred = grid_search.predict(X_valid)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 2min 4s
Wall time: 28min 26s
f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'criterion': 'gini', 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 1, 'n_estimators': 500}


CPU times: total: 2min 4s

Wall time: 28min 26s

f1_score на тестовом наборе: 0.81

Наилучшие значения гиперпараметров: {'criterion': 'gini', 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 1, 'n_estimators': 500}

## RANDOMIZEDSEARCHCV

In [17]:
random_search = model_selection.RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=random_state),
    param_distributions=param_grid,
    cv=5,
    n_iter=20,
    n_jobs=-1
)

%time random_search.fit(X_train, y_train) 
y_test_pred = random_search.predict(X_valid)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: total: 58.1 s
Wall time: 4min 42s
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 250, 'min_samples_leaf': 3, 'max_features': None, 'max_depth': 8, 'criterion': 'gini'}


CPU times: total: 58.1 s

Wall time: 4min 42s

f1_score на тестовом наборе: 0.82

Наилучшие значения гиперпараметров: {'n_estimators': 250, 'min_samples_leaf': 3, 'max_features': None, 'max_depth': 8, 'criterion': 'gini'}

## HYPEROPT

In [28]:
space = {
    'n_estimators': hp.randint('n_estimators', 500),
    'criterion':  hp.choice('criterion', ['gini', 'entropy']),
    'max_depth': hp.randint('max_depth', 12),
    'min_samples_leaf': hp.randint('min_samples_leaf', 10),
    'max_features': hp.choice('max_features', ['sqrt', 'log2', None])
}

In [29]:
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    if params['max_features'] is None:
        max_features = None
    else:
        max_features = str(params['max_features'])

    params = {
              'n_estimators': int(params['n_estimators']) + 1,
              'criterion': str(params['criterion']),
              'max_depth': int(params['max_depth']) + 1,
              'min_samples_leaf': int(params['min_samples_leaf']) + 1,
              'max_features': max_features
              }
  
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
    
    # применим  cross validation с тем же количеством фолдов
    score = model_selection.cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return -score

In [30]:
%%time

trials_RF = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials_RF, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 20/20 [03:00<00:00,  9.02s/trial, best loss: -0.8088187983870185]
Наилучшие значения гиперпараметров {'criterion': 0, 'max_depth': 9, 'max_features': 2, 'min_samples_leaf': 0, 'n_estimators': 238}
CPU times: total: 703 ms
Wall time: 3min


Наилучшие значения гиперпараметров {'criterion': 0, 'max_depth': 9, 'max_features': 2, 'min_samples_leaf': 0, 'n_estimators': 238}

>'min_samples_leaf': 0

будет взят 1 при вычислнии метрики

CPU times: total: 703 ms

Wall time: 3min

In [32]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state,
    n_estimators=int(best['n_estimators']),
    # criterion=str(best['criterion']),
    criterion='gini',
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf']) + 1,
    # max_features=str(best['max_features'])
    max_features=None
)
model.fit(X_train, y_train)
y_valid_pred = model.predict(X_valid)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))

f1_score на тестовом наборе: 0.82


f1_score на тестовом наборе: 0.82

## OPTUNA

In [3]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 50, 500)
  criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
  max_depth = trial.suggest_int('max_depth', 1, 12)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
  max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          criterion=criterion,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          max_features=max_features)
  
  kf = model_selection.KFold(n_splits=5)

  cv_metrics = model_selection.cross_validate(
    estimator=model,
    X=X_train,
    y=y_train,
    cv=kf,
    scoring='f1',
    return_train_score=True
  )

  return np.mean(cv_metrics['test_score'])

In [4]:
%%time

study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
study.optimize(optuna_rf, n_trials=10)

[I 2024-02-29 22:35:12,555] A new study created in memory with name: RandomForestClassifier
[I 2024-02-29 22:35:14,921] Trial 0 finished with value: 0.7266017904440351 and parameters: {'n_estimators': 242, 'criterion': 'entropy', 'max_depth': 1, 'min_samples_leaf': 2, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7266017904440351.
[I 2024-02-29 22:35:19,318] Trial 1 finished with value: 0.7320319760951876 and parameters: {'n_estimators': 464, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 4, 'max_features': 'log2'}. Best is trial 1 with value: 0.7320319760951876.
[I 2024-02-29 22:35:21,032] Trial 2 finished with value: 0.7366465752315627 and parameters: {'n_estimators': 168, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 10, 'max_features': 'log2'}. Best is trial 2 with value: 0.7366465752315627.
[I 2024-02-29 22:41:18,310] Trial 3 finished with value: 0.801929995784256 and parameters: {'n_estimators': 393, 'criterion': 'gini', 'max_depth': 9, 'min_sampl

CPU times: total: 18min 46s
Wall time: 18min 47s


[I 2024-02-29 22:35:14,921] Trial 0 finished with value: 0.7266017904440351 and parameters: {'n_estimators': 242, 'criterion': 'entropy', 'max_depth': 1, 'min_samples_leaf': 2, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7266017904440351.

[I 2024-02-29 22:35:19,318] Trial 1 finished with value: 0.7320319760951876 and parameters: {'n_estimators': 464, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 4, 'max_features': 'log2'}. Best is trial 1 with value: 0.7320319760951876.

[I 2024-02-29 22:35:21,032] Trial 2 finished with value: 0.7366465752315627 and parameters: {'n_estimators': 168, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 10, 'max_features': 'log2'}. Best is trial 2 with value: 0.7366465752315627.

[I 2024-02-29 22:41:18,310] Trial 3 finished with value: 0.801929995784256 and parameters: {'n_estimators': 393, 'criterion': 'gini', 'max_depth': 9, 'min_samples_leaf': 8, 'max_features': None}. Best is trial 3 with value: 0.801929995784256.

[I 2024-02-29 22:41:20,910] Trial 4 finished with value: 0.7910372049983344 and parameters: {'n_estimators': 69, 'criterion': 'gini', 'max_depth': 11, 'min_samples_leaf': 8, 'max_features': 'sqrt'}. Best is trial 3 with value: 0.801929995784256.

[I 2024-02-29 22:41:25,142] Trial 5 finished with value: 0.7301890736232322 and parameters: {'n_estimators': 368, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 10, 'max_features': 'sqrt'}. Best is trial 3 with value: 0.801929995784256.

[I 2024-02-29 22:41:26,845] Trial 6 finished with value: 0.7296883445184909 and parameters: {'n_estimators': 100, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 2, 'max_features': 'sqrt'}. Best is trial 3 with value: 0.801929995784256.

[I 2024-02-29 22:41:29,977] Trial 7 finished with value: 0.7369304063031659 and parameters: {'n_estimators': 184, 'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 3 with value: 0.801929995784256.

[I 2024-02-29 22:51:33,866] Trial 8 finished with value: 0.8071607365576604 and parameters: {'n_estimators': 498, 'criterion': 'entropy', 'max_depth': 11, 'min_samples_leaf': 4, 'max_features': None}. Best is trial 8 with value: 0.8071607365576604.

[I 2024-02-29 22:54:00,083] Trial 9 finished with value: 0.8031938619275596 and parameters: {'n_estimators': 130, 'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 6, 'max_features': None}. Best is trial 8 with value: 0.8071607365576604.

CPU times: total: 18min 46s

Wall time: 18min 47s

In [5]:
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

model = ensemble.RandomForestClassifier(**study.best_params, random_state=random_state)
model.fit(X_train, y_train)
y_valid_pred = model.predict(X_valid)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_valid, y_valid_pred)))

Наилучшие значения гиперпараметров {'n_estimators': 498, 'criterion': 'entropy', 'max_depth': 11, 'min_samples_leaf': 4, 'max_features': None}
f1_score на обучающем наборе: 0.81
f1_score на тестовом наборе: 0.82


Наилучшие значения гиперпараметров {'n_estimators': 498, 'criterion': 'entropy', 'max_depth': 11, 'min_samples_leaf': 4, 'max_features': None}

f1_score на обучающем наборе: 0.81

f1_score на тестовом наборе: 0.82

## Случайный лес - Итоги

GRIDSEARCHCV и OPTUNA относительно долго искали оптимальные параметры (30 и 20 минут при n_trials = 10) и при этом и значение метрики - 0.81. RANDOMIZEDSEARCHCV и hyperport по сравнению с предыдущеми нашли оптимальные параметры невероятно быстро, при этом hyperport запускался 2 раза, RANDOMIZEDSEARCHCV 1 раз. Оба способа показали метрику 0.82

Подводя итог, можно сказать, что GRIDSEARCHCV подходит в тех ситуациях, когда большинство параметров - категориальные, и не подходит на больших интервалах числовых параметрах. При этом метрика получается не лучшей, но и неплохой.

RANDOMIZEDSEARCHCV показал себя хорошо на обеих моделях по скорости и по метрике.

HYPEROPT подходит для поиска оптимальных числовых параметрах, но и с категориальными ведёт себя неплохо. Большой плюс, что видно процесс оценки модели во время работы кода. При этом самая высокая метрика среди всех способов. Мой выбор - HYPEROPT.

OPTUNA оказался медленным методом для логистической регрессии и очень медленным для случайного леса. Метрика сравнима с GRIDSEARCHCV.